In [46]:
%load_ext autoreload
%autoreload 2
from dataset_container import DataSet
import logging
import gymnasium as gym
from typing import List
from environment import MarketEnv
import constants

import json, glob, os, pickle, datetime
from visualizer import Visualizer
from util import extract_from_pickle
import os 
from pytorch_lightning import loggers
import ppo
from constants import *
from pytorch_lightning import Trainer

logging.basicConfig(level=logging.DEBUG)

PICKLES_PATH = os.path.join('..', 'data', "pickles")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
datasets = list(glob.glob(os.path.join(PICKLES_PATH, "*.pkl")))

datasets: List[DataSet] = list([extract_from_pickle(dataset) for dataset in datasets])

assert all(isinstance(dataset, DataSet) for dataset in datasets)


In [48]:
train_sets = []
validation_sets = []
test_sets = []

for dataset in datasets:

    train_set, validation_set, test_set = dataset.train_validation_test_split(datetime.datetime(2024, 1, 1), datetime.datetime(2024, 7, 1))
    train_sets.append(train_set)
    validation_sets.append(validation_set)
    test_sets.append(test_set)
    print(len(train_set), len(validation_set), len(test_set))



17475 7916 6546
17305 7906 6548
16907 7882 6567
7503 3458 2755
17442 7928 6569
9424 4239 3642
11695 5076 4051


In [49]:
configs = {
    "MAX_BUY_LIMIT": constants.MAX_BUY_LIMIT,
    "MEAN_RUN_DURATION": constants.MEAN_RUN_DURATION,
    "START_BALANCE": constants.START_BALANCE,
    "PREVIOUS_DATA_POINTS_AMOUNT": constants.PREVIOUS_DATA_POINTS_AMOUNT,
    "STD_RUN_DURATION": constants.STD_RUN_DURATION
}

gym.register("MarketEnv-v0", entry_point=MarketEnv)
env = gym.make("MarketEnv-v0", datasets=[train_sets[0]], config=configs)

c:\Python311\Lib\site-packages\gymnasium\envs\registration.py:642: UserWarning: WARN: Overriding environment MarketEnv-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


In [50]:
# Define the model
model = ppo.PPO(env=env, prev_data_points=PREVIOUS_DATA_POINTS_AMOUNT)

# Define the trainer
trainer = Trainer(max_epochs=100, accelerator="gpu")

# Train the model
trainer.fit(model)

--- Logging error ---
Traceback (most recent call last):
  File "c:\Python311\Lib\logging\__init__.py", line 1110, in emit
    msg = self.format(record)
          ^^^^^^^^^^^^^^^^^^^
  File "c:\Python311\Lib\logging\__init__.py", line 953, in format
    return fmt.format(record)
           ^^^^^^^^^^^^^^^^^^
  File "c:\Python311\Lib\logging\__init__.py", line 687, in format
    record.message = record.getMessage()
                     ^^^^^^^^^^^^^^^^^^^
  File "c:\Python311\Lib\logging\__init__.py", line 377, in getMessage
    msg = msg % self.args
          ~~~~^~~~~~~~~~~
TypeError: not all arguments converted during string formatting
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\Daniel\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Python311\Lib\site-packages\traitlets\config\application.py", line 1075, in laun

Training: |          | 0/? [00:00<?, ?it/s]

DEBUG:fsspec.local:open file: c:/Users/Daniel/OneDrive - TU Wien/Uni/7. Semester/ADL/repo/adl/src/lightning_logs/version_13/checkpoints/epoch=0-step=8.ckpt
DEBUG:fsspec.local:open file: c:/Users/Daniel/OneDrive - TU Wien/Uni/7. Semester/ADL/repo/adl/src/lightning_logs/version_13/checkpoints/epoch=1-step=16.ckpt
DEBUG:fsspec.local:open file: c:/Users/Daniel/OneDrive - TU Wien/Uni/7. Semester/ADL/repo/adl/src/lightning_logs/version_13/checkpoints/epoch=2-step=24.ckpt
DEBUG:fsspec.local:open file: c:/Users/Daniel/OneDrive - TU Wien/Uni/7. Semester/ADL/repo/adl/src/lightning_logs/version_13/checkpoints/epoch=3-step=32.ckpt
DEBUG:fsspec.local:open file: c:/Users/Daniel/OneDrive - TU Wien/Uni/7. Semester/ADL/repo/adl/src/lightning_logs/version_13/checkpoints/epoch=4-step=40.ckpt
DEBUG:fsspec.local:open file: c:/Users/Daniel/OneDrive - TU Wien/Uni/7. Semester/ADL/repo/adl/src/lightning_logs/version_13/checkpoints/epoch=5-step=48.ckpt
DEBUG:fsspec.local:open file: c:/Users/Daniel/OneDrive - TU

In [ ]:
import pprint
# Load the trained model
CHECKPOINT_PATH = r"C:\Users\Daniel\OneDrive - TU Wien\Uni\7. Semester\ADL\repo\adl\src\lightning_logs\version_13\checkpoints\epoch=99-step=800.ckpt"
model = ppo.PPO.load_from_checkpoint(CHECKPOINT_PATH, env=env, prev_data_points=PREVIOUS_DATA_POINTS_AMOUNT, device="cpu")

# Play the game with render mode set to 'human'
state = env.reset()
done = False
while not done:
    pprint.pprint("STATE:")
    pprint.pprint(state)
    action = model.make_move(state, "cpu")
    pprint.pprint("ACTION:")
    pprint.pprint(action)
    state, reward, done, info = env.step(action)
    pprint.pprint("REWARD:")
    pprint.pprint(reward)
    state = (state,info)

DEBUG:fsspec.local:open file: C:/Users/Daniel/OneDrive - TU Wien/Uni/7. Semester/ADL/repo/adl/src/lightning_logs/version_13/checkpoints/epoch=999-step=8000.ckpt


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/Daniel/OneDrive - TU Wien/Uni/7. Semester/ADL/repo/adl/src/lightning_logs/version_13/checkpoints/epoch=999-step=8000.ckpt'